# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [3]:
# Import the data
data = pd.read_csv('loan_final.csv')

# Drop rows with no target value
data.dropna(subset=['total_pymnt'], axis=0, inplace=True)

# Print the first five rows
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [4]:
data.shape

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [6]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [8]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer()

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_imputed), columns=cont_features)

# Transform test data
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_imputed), columns=cont_features)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [21]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat.fillna(value='missing', inplace=True)
X_test_cat.fillna(value='missing', inplace=True)

# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe.todense(), columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe.todense(), columns=cat_columns)

Run the below cell to combine the numeric and categorical features. 

In [22]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [24]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [25]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [26]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model, epochs=150, batch_size=256, verbose=0)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [27]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled, cv=5)

- Find the RMSE on train data 

In [28]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_baseline_preds))

0.44420579384148773

- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [31]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
np.sqrt(mean_squared_error(baseline_preds, y_train))

4046.0660261887274

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [32]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
     # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(50, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(25, activation='relu'))
    
    # Third hidden layer
    model.add(layers.Dense(10, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

In [33]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(create_bigger_model, epochs=300, batch_size=256, verbose=1)

In [34]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2, X_train_all, y_train_scaled, cv=5)

Epoch 1/300
23819/23819 [==============================] - 0s 14us/step - loss: 0.4480 - mse: 0.4480
Epoch 2/300
23819/23819 [==============================] - 0s 11us/step - loss: 0.2331 - mse: 0.2331
Epoch 3/300
23819/23819 [==============================] - 0s 11us/step - loss: 0.2212 - mse: 0.2212
Epoch 4/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.2161 - mse: 0.2161
Epoch 5/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.2125 - mse: 0.2125
Epoch 6/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.2101 - mse: 0.2101
Epoch 7/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.2082 - mse: 0.2082
Epoch 8/300
23819/23819 [==============================] - 0s 11us/step - loss: 0.2066 - mse: 0.2066
Epoch 9/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.2052 - mse: 0.2052
Epoch 10/300
23819/23819 [==============================] - 0s 12us/step - loss: 0.2042 - ms

23819/23819 [==============================] - 0s 11us/step - loss: 0.1882 - mse: 0.1882
Epoch 82/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1882 - mse: 0.1882
Epoch 83/300
23819/23819 [==============================] - 0s 13us/step - loss: 0.1880 - mse: 0.1880
Epoch 84/300
23819/23819 [==============================] - 0s 14us/step - loss: 0.1879 - mse: 0.1879
Epoch 85/300
23819/23819 [==============================] - 0s 12us/step - loss: 0.1879 - mse: 0.1879
Epoch 86/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.1879 - mse: 0.1879
Epoch 87/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.1878 - mse: 0.1878
Epoch 88/300
23819/23819 [==============================] - 0s 11us/step - loss: 0.1876 - mse: 0.1876: 0s - loss: 0.1974 - mse: 0.
Epoch 89/300
23819/23819 [==============================] - 0s 11us/step - loss: 0.1876 - mse: 0.1876
Epoch 90/300
23819/23819 [==============================] - 0s 11us

23819/23819 [==============================] - 0s 6us/step - loss: 0.1847 - mse: 0.1847
Epoch 162/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1844 - mse: 0.1844
Epoch 163/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1844 - mse: 0.1844
Epoch 164/300
23819/23819 [==============================] - 0s 5us/step - loss: 0.1843 - mse: 0.1843
Epoch 165/300
23819/23819 [==============================] - 0s 5us/step - loss: 0.1842 - mse: 0.1842
Epoch 166/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.1841 - mse: 0.1841
Epoch 167/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1842 - mse: 0.1842
Epoch 168/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1842 - mse: 0.1842
Epoch 169/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1840 - mse: 0.1840
Epoch 170/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1840 - mse: 

23819/23819 [==============================] - 0s 8us/step - loss: 0.1821 - mse: 0.1821
Epoch 241/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1823 - mse: 0.1823
Epoch 242/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.1824 - mse: 0.1824
Epoch 243/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1821 - mse: 0.1821
Epoch 244/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.1823 - mse: 0.1823
Epoch 245/300
23819/23819 [==============================] - 0s 10us/step - loss: 0.1823 - mse: 0.1823
Epoch 246/300
23819/23819 [==============================] - 0s 9us/step - loss: 0.1824 - mse: 0.1824
Epoch 247/300
23819/23819 [==============================] - 0s 9us/step - loss: 0.1821 - mse: 0.1821
Epoch 248/300
23819/23819 [==============================] - 0s 14us/step - loss: 0.1819 - mse: 0.1819
Epoch 249/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1822 - ms

23819/23819 [==============================] - 0s 12us/step - loss: 0.2012 - mse: 0.2012
Epoch 20/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.2009 - mse: 0.2009
Epoch 21/300
23819/23819 [==============================] - 0s 9us/step - loss: 0.2002 - mse: 0.2002
Epoch 22/300
23819/23819 [==============================] - 0s 9us/step - loss: 0.2001 - mse: 0.2001
Epoch 23/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1998 - mse: 0.1998
Epoch 24/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1996 - mse: 0.1996
Epoch 25/300
23819/23819 [==============================] - ETA: 0s - loss: 0.2016 - mse: 0.201 - 0s 8us/step - loss: 0.1999 - mse: 0.1999
Epoch 26/300
23819/23819 [==============================] - 0s 9us/step - loss: 0.1991 - mse: 0.1991
Epoch 27/300
23819/23819 [==============================] - 0s 9us/step - loss: 0.1988 - mse: 0.1988
Epoch 28/300
23819/23819 [==============================] - 0s 10

...

23819/23819 [==============================] - 0s 8us/step - loss: 0.1840 - mse: 0.1840
Epoch 214/300
23819/23819 [==============================] - 0s 7us/step - loss: 0.1839 - mse: 0.1839
Epoch 215/300
23819/23819 [==============================] - 0s 7us/step - loss: 0.1841 - mse: 0.1841
Epoch 216/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1840 - mse: 0.1840
Epoch 217/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1841 - mse: 0.1841
Epoch 218/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1844 - mse: 0.1844
Epoch 219/300
23819/23819 [==============================] - 0s 7us/step - loss: 0.1841 - mse: 0.1841
Epoch 220/300
23819/23819 [==============================] - 0s 7us/step - loss: 0.1840 - mse: 0.1840
Epoch 221/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1847 - mse: 0.1847
Epoch 222/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1839 - mse: 0

23819/23819 [==============================] - 0s 8us/step - loss: 0.1825 - mse: 0.1825
Epoch 294/300
23819/23819 [==============================] - 0s 7us/step - loss: 0.1825 - mse: 0.1825
Epoch 295/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1824 - mse: 0.1824
Epoch 296/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1824 - mse: 0.1824
Epoch 297/300
23819/23819 [==============================] - 0s 7us/step - loss: 0.1822 - mse: 0.1822
Epoch 298/300
23819/23819 [==============================] - 0s 6us/step - loss: 0.1821 - mse: 0.1821
Epoch 299/300
23819/23819 [==============================] - 0s 8us/step - loss: 0.1823 - mse: 0.1823
Epoch 300/300
5955/5955 [==============================] - 0s 23us/step
Epoch 1/300
23820/23820 [==============================] - 0s 14us/step - loss: 0.7222 - mse: 0.7222
Epoch 2/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.2514 - mse: 0.2514
Epoch 3/300
23820/23820 [==

23820/23820 [==============================] - 0s 9us/step - loss: 0.1920 - mse: 0.1920
Epoch 74/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1917 - mse: 0.1917
Epoch 75/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1917 - mse: 0.1917
Epoch 76/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1915 - mse: 0.1915
Epoch 77/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1917 - mse: 0.1917
Epoch 78/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1914 - mse: 0.1914
Epoch 79/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1914 - mse: 0.1914
Epoch 80/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1913 - mse: 0.1913
Epoch 81/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1914 - mse: 0.1914
Epoch 82/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1909 - mse: 0.1909
Epo

23820/23820 [==============================] - 0s 8us/step - loss: 0.1873 - mse: 0.1873
Epoch 154/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1872 - mse: 0.1872
Epoch 155/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1872 - mse: 0.1872
Epoch 156/300
23820/23820 [==============================] - 0s 6us/step - loss: 0.1871 - mse: 0.1871
Epoch 157/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1870 - mse: 0.1870
Epoch 158/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1870 - mse: 0.1870
Epoch 159/300
23820/23820 [==============================] - 0s 12us/step - loss: 0.1869 - mse: 0.1869
Epoch 160/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1868 - mse: 0.1868
Epoch 161/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1869 - mse: 0.1869
Epoch 162/300
23820/23820 [==============================] - 0s 6us/step - loss: 0.1869 - mse: 

23820/23820 [==============================] - 0s 9us/step - loss: 0.1849 - mse: 0.1849
Epoch 234/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1848 - mse: 0.1848
Epoch 235/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1848 - mse: 0.1848
Epoch 236/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1847 - mse: 0.1847
Epoch 237/300
23820/23820 [==============================] - 0s 9us/step - loss: 0.1848 - mse: 0.1848
Epoch 238/300
23820/23820 [==============================] - 0s 9us/step - loss: 0.1849 - mse: 0.1849
Epoch 239/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1848 - mse: 0.1848
Epoch 240/300
23820/23820 [==============================] - 0s 8us/step - loss: 0.1857 - mse: 0.1857
Epoch 241/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1847 - mse: 0.1847
Epoch 242/300
23820/23820 [==============================] - 0s 7us/step - loss: 0.1846 - mse: 0

In [35]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(cv_bigger_model_preds, y_train_scaled))

0.43680763544289997

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [36]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
    # Initialize model
    model = models.Sequential()
    model.add(layers.Dropout(.3, input_shape=(n_features,)))
    
    # First hidden layer
    model.add(layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.005)))
    model.add(layers.Dropout(.3))

    # Second hidden layer
    model.add(layers.Dense(25, activation='relu', kernel_regularizer=regularizers.l2(0.005)))
    model.add(layers.Dropout(.3))
    
    # Third hidden layer
    model.add(layers.Dense(10, activation='relu', kernel_regularizer=regularizers.l2(0.005)))
    model.add(layers.Dropout(.3))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

In [37]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model, epochs=150, batch_size=256, verbose=1)

In [38]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled, cv=5)

Epoch 1/150
23819/23819 [==============================] - 1s 21us/step - loss: 1.1545 - mse: 0.7579
Epoch 2/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.9662 - mse: 0.5802: 0s - loss: 0.9629 - mse: 0
Epoch 3/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.9124 - mse: 0.5356
Epoch 4/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.8868 - mse: 0.5182
Epoch 5/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.8598 - mse: 0.4991
Epoch 6/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.8314 - mse: 0.4781
Epoch 7/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.8120 - mse: 0.4657
Epoch 8/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.7995 - mse: 0.4602
Epoch 9/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.7715 - mse: 0.4389
Epoch 10/150
23819/23819 [==============================] - 0s 

23819/23819 [==============================] - 0s 14us/step - loss: 0.4442 - mse: 0.3502
Epoch 81/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4381 - mse: 0.3456
Epoch 82/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.4425 - mse: 0.3515
Epoch 83/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4361 - mse: 0.3465
Epoch 84/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.4276 - mse: 0.3394
Epoch 85/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4267 - mse: 0.3398
Epoch 86/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.4284 - mse: 0.3428
Epoch 87/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4295 - mse: 0.3452
Epoch 88/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.4344 - mse: 0.3514
Epoch 89/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4281 - mse: 

23819/23819 [==============================] - 0s 16us/step - loss: 0.8139 - mse: 0.4713
Epoch 10/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.7915 - mse: 0.4554
Epoch 11/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.7864 - mse: 0.4569
Epoch 12/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.7778 - mse: 0.4548
Epoch 13/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.7604 - mse: 0.4437
Epoch 14/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.7475 - mse: 0.4369
Epoch 15/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.7395 - mse: 0.4348
Epoch 16/150
23819/23819 [==============================] - 0s 17us/step - loss: 0.7302 - mse: 0.4314
Epoch 17/150
23819/23819 [==============================] - 0s 16us/step - loss: 0.7165 - mse: 0.4235
Epoch 18/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.7215 - mse: 

23819/23819 [==============================] - 0s 9us/step - loss: 0.4579 - mse: 0.3728
Epoch 90/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.4577 - mse: 0.3738
Epoch 91/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.4589 - mse: 0.3763
Epoch 92/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.4539 - mse: 0.3724
Epoch 93/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.4571 - mse: 0.3767
Epoch 94/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.4572 - mse: 0.3780
Epoch 95/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4432 - mse: 0.3651
Epoch 96/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4462 - mse: 0.3691
Epoch 97/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.4497 - mse: 0.3737
Epoch 98/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.4440 - mse: 0.3691
E

23819/23819 [==============================] - 0s 12us/step - loss: 0.6841 - mse: 0.4072
Epoch 19/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.6755 - mse: 0.4039
Epoch 20/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.6695 - mse: 0.4031
Epoch 21/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.6609 - mse: 0.3995
Epoch 22/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.6522 - mse: 0.3957
Epoch 23/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.6528 - mse: 0.4011
Epoch 24/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.6441 - mse: 0.3972
Epoch 25/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.6338 - mse: 0.3915
Epoch 26/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.6345 - mse: 0.3966
Epoch 27/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.6208 - mse: 

23819/23819 [==============================] - 0s 11us/step - loss: 0.4197 - mse: 0.3463
Epoch 99/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4138 - mse: 0.3414
Epoch 100/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.4233 - mse: 0.3519
Epoch 101/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4181 - mse: 0.3476
Epoch 102/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4088 - mse: 0.3394
Epoch 103/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.4134 - mse: 0.3449
Epoch 104/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4133 - mse: 0.3458
Epoch 105/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4099 - mse: 0.3433
Epoch 106/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4128 - mse: 0.3471
Epoch 107/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4118 

23819/23819 [==============================] - 0s 12us/step - loss: 0.6348 - mse: 0.3996
Epoch 28/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.6295 - mse: 0.3984
Epoch 29/150
23819/23819 [==============================] - 0s 20us/step - loss: 0.6189 - mse: 0.3920
Epoch 30/150
23819/23819 [==============================] - 0s 20us/step - loss: 0.6177 - mse: 0.3949
Epoch 31/150
23819/23819 [==============================] - 0s 15us/step - loss: 0.6115 - mse: 0.3928
Epoch 32/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.6034 - mse: 0.3887
Epoch 33/150
23819/23819 [==============================] - 0s 16us/step - loss: 0.6058 - mse: 0.3951
Epoch 34/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.5844 - mse: 0.3775
Epoch 35/150
23819/23819 [==============================] - 0s 15us/step - loss: 0.5886 - mse: 0.3854
Epoch 36/150
23819/23819 [==============================] - 0s 17us/step - loss: 0.5815 - mse: 

23819/23819 [==============================] - 0s 19us/step - loss: 0.4140 - mse: 0.3496
Epoch 107/150
23819/23819 [==============================] - 0s 16us/step - loss: 0.4145 - mse: 0.3509
Epoch 108/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4103 - mse: 0.3477
Epoch 109/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.4094 - mse: 0.3476
Epoch 110/150
23819/23819 [==============================] - 0s 19us/step - loss: 0.4158 - mse: 0.3547
Epoch 111/150
23819/23819 [==============================] - 0s 15us/step - loss: 0.4051 - mse: 0.3449
Epoch 112/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.4057 - mse: 0.3463
Epoch 113/150
23819/23819 [==============================] - 0s 15us/step - loss: 0.4059 - mse: 0.3472
Epoch 114/150
23819/23819 [==============================] - 1s 30us/step - loss: 0.4078 - mse: 0.3498
Epoch 115/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.408

23820/23820 [==============================] - 0s 9us/step - loss: 0.6469 - mse: 0.4497
Epoch 36/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.6392 - mse: 0.4455
Epoch 37/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.6270 - mse: 0.4369
Epoch 38/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.6386 - mse: 0.4520
Epoch 39/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.6281 - mse: 0.4447
Epoch 40/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.6119 - mse: 0.4317
Epoch 41/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.6105 - mse: 0.4335
Epoch 42/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.6178 - mse: 0.4439
Epoch 43/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.6165 - mse: 0.4458
Epoch 44/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.6102 - mse: 0.4425
Epo

23820/23820 [==============================] - 0s 13us/step - loss: 0.4590 - mse: 0.4008
Epoch 116/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.4681 - mse: 0.4107
Epoch 117/150
23820/23820 [==============================] - 0s 13us/step - loss: 0.4588 - mse: 0.4021
Epoch 118/150
23820/23820 [==============================] - 0s 13us/step - loss: 0.4679 - mse: 0.4119
Epoch 119/150
23820/23820 [==============================] - 0s 10us/step - loss: 0.4612 - mse: 0.4059
Epoch 120/150
23820/23820 [==============================] - 0s 10us/step - loss: 0.4625 - mse: 0.4078
Epoch 121/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.4646 - mse: 0.4105
Epoch 122/150
23820/23820 [==============================] - 0s 10us/step - loss: 0.4610 - mse: 0.4075
Epoch 123/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.4542 - mse: 0.4012
Epoch 124/150
23820/23820 [==============================] - 0s 10us/step - loss: 0.4551 

In [39]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(cv_dropout_preds, y_train_scaled))

0.5402670559090428

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [40]:
final_model = models.Sequential()
final_model.add(layers.Dense(50, activation='relu', input_shape=(n_features,)))
final_model.add(layers.Dense(25, activation='relu'))
final_model.add(layers.Dense(10, activation='relu'))
final_model.add(layers.Dense(1, activation='linear'))
final_model.compile(optimizer='SGD', loss='mse', metrics=['mse']) 
final_model.fit(X_train_all, y_train_scaled, epochs=300, batch_size=256)

Epoch 1/300
29774/29774 [==============================] - 0s 16us/step - loss: 0.3107 - mse: 0.3107
Epoch 2/300
29774/29774 [==============================] - 0s 10us/step - loss: 0.2248 - mse: 0.2248
Epoch 3/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.2173 - mse: 0.2173
Epoch 4/300
29774/29774 [==============================] - 0s 11us/step - loss: 0.2131 - mse: 0.2131
Epoch 5/300
29774/29774 [==============================] - 0s 9us/step - loss: 0.2104 - mse: 0.2104
Epoch 6/300
29774/29774 [==============================] - 0s 10us/step - loss: 0.2079 - mse: 0.2079
Epoch 7/300
29774/29774 [==============================] - 0s 10us/step - loss: 0.2063 - mse: 0.2063
Epoch 8/300
29774/29774 [==============================] - 0s 10us/step - loss: 0.2050 - mse: 0.2050
Epoch 9/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.2037 - mse: 0.2037
Epoch 10/300
29774/29774 [==============================] - 0s 10us/step - loss: 0.2030 - mse:

29774/29774 [==============================] - 0s 10us/step - loss: 0.1892 - mse: 0.1892
Epoch 82/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1891 - mse: 0.1891
Epoch 83/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1891 - mse: 0.1891
Epoch 84/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1891 - mse: 0.1891
Epoch 85/300
29774/29774 [==============================] - 0s 7us/step - loss: 0.1890 - mse: 0.1890
Epoch 86/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1888 - mse: 0.1888
Epoch 87/300
29774/29774 [==============================] - 0s 10us/step - loss: 0.1889 - mse: 0.1889
Epoch 88/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1887 - mse: 0.1887
Epoch 89/300
29774/29774 [==============================] - 0s 10us/step - loss: 0.1887 - mse: 0.1887
Epoch 90/300
29774/29774 [==============================] - 0s 9us/step - loss: 0.1887 - mse: 0.1887


Epoch 161/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1861 - mse: 0.1861
Epoch 162/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1859 - mse: 0.1859
Epoch 163/300
29774/29774 [==============================] - 0s 9us/step - loss: 0.1860 - mse: 0.1860
Epoch 164/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1860 - mse: 0.1860
Epoch 165/300
29774/29774 [==============================] - 0s 11us/step - loss: 0.1859 - mse: 0.1859
Epoch 166/300
29774/29774 [==============================] - 0s 10us/step - loss: 0.1860 - mse: 0.1860
Epoch 167/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1857 - mse: 0.1857
Epoch 168/300
29774/29774 [==============================] - 0s 11us/step - loss: 0.1858 - mse: 0.1858
Epoch 169/300
29774/29774 [==============================] - 0s 12us/step - loss: 0.1858 - mse: 0.1858
Epoch 170/300
29774/29774 [==============================] - 0s 10us/step - lo

29774/29774 [==============================] - 0s 12us/step - loss: 0.1842 - mse: 0.1842
Epoch 242/300
29774/29774 [==============================] - 0s 11us/step - loss: 0.1842 - mse: 0.1842
Epoch 243/300
29774/29774 [==============================] - 0s 6us/step - loss: 0.1840 - mse: 0.1840
Epoch 244/300
29774/29774 [==============================] - 0s 6us/step - loss: 0.1842 - mse: 0.1842
Epoch 245/300
29774/29774 [==============================] - 0s 12us/step - loss: 0.1842 - mse: 0.1842
Epoch 246/300
29774/29774 [==============================] - 0s 8us/step - loss: 0.1840 - mse: 0.1840
Epoch 247/300
29774/29774 [==============================] - 0s 7us/step - loss: 0.1841 - mse: 0.1841
Epoch 248/300
29774/29774 [==============================] - 0s 13us/step - loss: 0.1840 - mse: 0.1840
Epoch 249/300
29774/29774 [==============================] - 0s 7us/step - loss: 0.1842 - mse: 0.1842
Epoch 250/300
29774/29774 [==============================] - 0s 11us/step - loss: 0.1841 - m

In [41]:
y_preds_scaled = final_model.predict(X_test_all)
y_preds = ss_y.inverse_transform(y_preds_scaled)
np.sqrt(mean_squared_error(y_test, y_preds))

3847.6675591338408

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 